In the last notebook I was talking about making pairing operations more efficient.  But the pairing operations were using an "id" that I left unexplained.  This "id" was a pretty nifty optimization so it deserves it's own explanation.

For the [transit-score heatmaps](https://johnfwhitesell.github.io/transit/), I was matching addresses to shapes in order to produce averages over the shape.  Originally I was doing this with geometric calculations but this was very slow, every point needed to be checked against shapes until it found one that it fell inside.  With smaller shapes, many more checks were needed until a match so this could get very slow indeed.

In order to speed things up, I decided to put everything within a standardized grid.  By doing this I could [calculate the hex](https://www.redblobgames.com/grids/hexagons/#pixel-to-hex) from a position rather then doing any lookup.  This calculation process is very rapid.  I originally had a tqdm progressbar to track how long the assigments were taking.  Once I switched over from shapes to calculations my progress bar rounded down to 0 seconds because my computer could calculate tens of thousands of pair combinations a second and I only had 20,000 addresses to feed into the model.

The downside to this is just calculating the correct hex doesn't modify that hex, you still have to tell the computer to look up the object you want to modify.  However this too can be greatly optimized.

Every hexogon axis had a set of grid coordinates ([cubic coordinates](https://www.redblobgames.com/grids/hexagons/#coordinates)).  Once I calculated those I could just look up which object in my pandas dataframe matched those coordinates:

In [ ]:
%timeit test['grid']==(25,25,25)
# 1.24 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
# note: I just used 25, 25, 25 for speed testing purposes, those coordinates aren't actually possible in a cubic hex grid


Checking against every object in 1.24 miliseconds is a big improvement over geometric matching but it's actually extremely slow compared to what is possible.  To improve this, I replaced those coordinates with a single value:

In [ ]:
index_key = lambda t, m: (t[0]+m+1 + (t[1]+m+1)*2*m)

I did two things here.  First of all I ditched the third coordinate because it wasn't necessary to make a unique key.  (In cubic hex coordinates x+y+z=0 for every hex).  I took the range of hexes and the grid location and calculated an interger that was unique for every location.  To understand how this works, imagine you have a set of x-y coordinates where every x and every y is a value between 1 and 9.  You can easily represent this as a single number by just multiplying the y by 10 and adding them together.  For instance x=5, y=4 could be represented as x,y = 45.  For me the range wasn't 1-9 it was m to -m.  As a result I made all the values positive and multiplied the y values by 2*m instead of multiplying them by 10.

Looking up a single integer was far faster then looking up three integers:

In [ ]:
%timeit test['integer_form']== 2550
# 133 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Now we are cooking with kerosene!

Of course the really big advantage of this matching system is it's scalability compared to the sort operations I was doing before.  With the geometric merging, every time you added another shape to the map, you were potentially adding another task to every address in the list.  With this the bulk of the computing time is being spent on calculating the keys, a task which is virtually agnostic towards the size of the other dataset.  This means there is very little cost to greatly increasing the granularity of the hexes, or adding more addresses.  It just adds a linear cost.

Of course if the number of hexagons got truly enormous the speed costs would stop looking quite so linear because the actual merging would get slower.  In that case it might make sense to create a bit comparison operation instead of an integer comparison.  Right now that's really not necessary because the integer comparison is less then 10% of the processing time for all three tasks.  More hexes would also probably mean a larger geometric area, in which case it would start to make sense to create buffers of individualized regions, i.e. create a subgrid for each city that way you are looking at a short list of hexes within that city instead of a long list of hexes all across the state.  However these are somewhat missing the forest for the trees.  The big takeaway I took from this project is that if you have a very slow task you can think about ways to replace it with a completely different task.  This is what made it possible to improve things by an order of magnitude then improve them by an order of magnitude all over again.  General solutions might be nice but specific solutions can be a heck of a lot faster.